In [1]:
!pip install pdfplumber

     ---------------------------------------- 60.2/60.2 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 5.6/5.6 MB 18.9 MB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 14.5 MB/s eta 0:00:00


In [2]:
import pdfplumber
import pandas as pd
import re


In [22]:
rows = []
with pdfplumber.open(r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Revosax\PsychKHEinzugsgebietsVO-Anlagen.pdf") as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row and len(row) >= 3:
                    rows.append(row[:3])  # nur die ersten 3 Spalten
                    
                    

In [30]:
rows = []
with pdfplumber.open(r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Revosax\PsychKHEinzugsgebietsVO-Anlagen.pdf") as pdf:
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row and len(row) >= 3:
                    rows.append(row[:3])  # nur die ersten 3 Spalten

# Abschnitte trennen
marker_erwachsene = "Einzugsgebiete für die Versorgung der Erwachsenen"
marker_kinder = "Einzugsgebiete für die Versorgung der Kinder und Jugendlichen"

rows_erwachsene = []
rows_kinder = []

current_section = None

for row in rows:
    row_text = " ".join([str(cell) if cell else "" for cell in row]).lower()
    if marker_erwachsene.lower() in row_text:
        current_section = "erwachsene"
        continue
    elif marker_kinder.lower() in row_text:
        current_section = "kinder"
        continue
    
    if current_section == "erwachsene":
        rows_erwachsene.append(row)
    elif current_section == "kinder":
        rows_kinder.append(row)

# Jetzt hast du getrennte Daten für beide Abschnitte


In [33]:
import pdfplumber
import re
import pandas as pd

def is_gemeindeschluessel(s):
    # Prüft, ob ein String eine 8-stellige Zahl enthält (Gemeindeschlüssel)
    return bool(re.match(r'^\d{8}$', s.strip()))

def parse_pdf(path):
    erwachsene_data = []
    kinder_data = []
    current_section = None
    current_hospital = None
    current_landkreis = None
    
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if not text:
                continue

            lines = text.split('\n')

            for line in lines:
                line = line.strip()
                # Abschnittswechsel erkennen
                if "Einzugsgebiete für die Versorgung der Erwachsenen" in line:
                    current_section = 'erwachsene'
                    continue
                if "Einzugsgebiete für die Versorgung der Kinder und Jugendlichen" in line:
                    current_section = 'kinder'
                    continue
                
                # Kopfzeilen ignorieren
                if any(h in line.lower() for h in ['versorgungsverpflichtetes', 'landkreis', 'stadtteil', 'gemeindeschlüssel', 'krankenhaus']):
                    continue
                
                # Krankenhauszeile (erkennt man daran, dass sie kein GKZ am Anfang hat)
                # Beispiel: "Heinrich-Braun-Klinikum gGmbH Landkreis Zwickau"
                if not re.match(r'^\d{8}', line) and len(line) > 5:
                    # Krankenhaus und Landkreis trennen, z.B. am Wort "Landkreis"
                    parts = re.split(r'\s+Landkreis\s+|\s+Kreisfreie Stadt\s+', line)
                    current_hospital = parts[0].strip()
                    current_landkreis = parts[1].strip() if len(parts) > 1 else None
                    continue
                
                # Zeile mit Gemeindeschlüssel + Ort, z.B. "14522020 Augustusburg"
                m = re.match(r'^(\d{8})\s+(.+)$', line)
                if m:
                    gkz = m.group(1)
                    ort = m.group(2)
                    row = {
                        'Krankenhaus': current_hospital,
                        'Landkreis': current_landkreis,
                        'Gemeindeschlüssel': gkz,
                        'Ort': ort
                    }
                    if current_section == 'erwachsene':
                        erwachsene_data.append(row)
                    elif current_section == 'kinder':
                        kinder_data.append(row)
                    continue
                
                # Manchmal stehen Zeilen mit nur Ort ohne GKZ? Wenn ja, kannst du sie optional behandeln.

    df_erwachsene = pd.DataFrame(erwachsene_data)
    df_kinder = pd.DataFrame(kinder_data)
    return df_erwachsene, df_kinder

# Beispiel-Aufruf
df_erwachsene, df_kinder = parse_pdf(r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Revosax\PsychKHEinzugsgebietsVO-Anlagen.pdf")

print("Erwachsene:")
print(df_erwachsene.head())

print("\nKinder und Jugendliche:")
print(df_kinder.head())


Erwachsene:
                  Krankenhaus Landkreis Gemeindeschlüssel            Ort
0  (zu § 1 Absatz 1 Nummer 1)      None          14522020   Augustusburg
1  (zu § 1 Absatz 1 Nummer 1)      None          14522060      Burgstädt
2  (zu § 1 Absatz 1 Nummer 1)      None          14522110      Eppendorf
3  (zu § 1 Absatz 1 Nummer 1)      None          14522140          Flöha
4  (zu § 1 Absatz 1 Nummer 1)      None          14522260  Hartmannsdorf

Kinder und Jugendliche:
                           Krankenhaus Landkreis Gemeindeschlüssel  \
0  Standort Zwickau 14524010 Bernsdorf      None          14524020   
1  Standort Zwickau 14524010 Bernsdorf      None          14524030   
2  Standort Zwickau 14524010 Bernsdorf      None          14524040   
3  Standort Zwickau 14524010 Bernsdorf      None          14524050   
4  Standort Zwickau 14524010 Bernsdorf      None          14524060   

            Ort  
0    Callenberg  
1  Crimmitschau  
2   Crinitzberg  
3    Dennheritz  
4     Fraureut

In [34]:
df_erwachsene

,Krankenhaus,Landkreis,Gemeindeschlüssel,Ort
0,(zu § 1 Absatz 1 Nummer 1),None,14522020,Augustusburg
1,(zu § 1 Absatz 1 Nummer 1),None,14522060,Burgstädt
2,(zu § 1 Absatz 1 Nummer 1),None,14522110,Eppendorf
3,(zu § 1 Absatz 1 Nummer 1),None,14522140,Flöha
4,(zu § 1 Absatz 1 Nummer 1),None,14522260,Hartmannsdorf
...,...,...,...,...
410,Hubertusburg gGmbH 14627030 Ebersbach,None,14730210,Naundorf
411,Hubertusburg gGmbH 14627030 Ebersbach,None,14730230,Oschatz
412,Hubertusburg gGmbH 14627030 Ebersbach,None,14730310,Torgau
413,Hubertusburg gGmbH 14627030 Ebersbach,None,14730320,Trossin


In [28]:
import pandas as pd
import re

# Beispiel: deine geladene Tabelle
df = pd.DataFrame(rows)

# Überschriften-Zeilen herausfiltern
kopfzeilen = [
    'versorgungsverpflichtetes\nKrankenhaus',
    'Landkreis/Kreisfreie Stadt\nGemeindeschlüssel',
    'Stadt/Stadtteil/Gemeinde'
]
df_clean = df[~df[0].isin(kopfzeilen)]

# Ergebnisse speichern
result = []

# Gehe jede Zeile durch
for row in df_clean.itertuples(index=False):
    krankenhaus = row[0].strip()
    zweite_spalte = str(row[1])
    
    # Alle 8-stelligen Zahlen (Gemeindeschlüssel) extrahieren
    gemeindeschluessel_liste = re.findall(r'\b\d{8}\b', zweite_spalte)

    # Für jeden Gemeindeschlüssel eine neue Zeile speichern
    for gkz in gemeindeschluessel_liste:
        result.append({'Krankenhaus': krankenhaus, 'Gemeindeschlüssel': gkz})

# DataFrame ausgeben
df_result = pd.DataFrame(result)
df_result

df_result.to_csv(r"C:\Users\agnes\Documents\EAGLE\Innovation_Lab\Daten\Revosax\processed\KHEinzugsgebietsVO-Anlagen.csv", index=False)
